In [2]:
import boto3


s3_resource = boto3.resource("s3")
print("Hello, Amazon S3! Let's list your buckets:")
for bucket in s3_resource.buckets.all():
    print(f"\t{bucket.name}")

Hello, Amazon S3! Let's list your buckets:
	video-aws-bucket


In [2]:
import boto3

# Initialize a session using Amazon S3
s3 = boto3.client('s3')

# File to be uploaded
file_name = "images/poatan.png"
bucket_name = 'video-aws-bucket'
object_name = 'image_0.png' 

# Upload the file
try:
    response = s3.upload_file(file_name, bucket_name, object_name)
    print("File uploaded successfully")
except Exception as e:
    print(f"Error uploading file: {e}")


File uploaded successfully


## Upload Video Frames to AWS

When uploading to the bucket, we're going to want to make sure that the same file name is not duplicated. In order to do this, we're going to maintain a key-value store containing video names and metadata

In [5]:
import boto3
import time
from botocore.exceptions import ClientError

def upload_metadata(video_name, table):
    try:
        # Check if video_name already exists
        response = table.get_item(Key={'video_name': video_name})
        if 'Item' in response:
            raise ValueError(f"Video name '{video_name}' already exists.")
        
        table.put_item(
            Item={
                'video_name': video_name,
                'timestamp': int(time.time())
            }
        )
    except ClientError as e:
        print(e.response['Error']['Message'])
        return False
    except ValueError as e:
        print(e)
        return False
    
    return True

# Example use
video_name = "example_video"
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('video_names')
unique_id = upload_metadata(video_name, table)
if unique_id:
    print(f"Metadata uploaded with unique ID: {unique_id}")
else:
    print("Failed to upload metadata.")

Metadata uploaded with unique ID: True
